In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense

# -------------------------------
# 1. Training Text (multiple words or full sentence)
# -------------------------------
training_text = "hello how are you"

# -------------------------------
# 2. Build Character Mapping
# -------------------------------
chars = sorted(list(set(training_text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for ch, i in char_to_idx.items()}
vocab_size = len(chars)

# -------------------------------
# 3. Prepare Sequence Data
# -------------------------------
sequence_length = 10  # window size
step = 1

input_sequences = []
target_sequences = []

for i in range(0, len(training_text) - sequence_length):
    input_chunk = training_text[i: i + sequence_length]
    target_chunk = training_text[i + 1: i + sequence_length + 1]
    input_sequences.append([char_to_idx[ch] for ch in input_chunk])
    target_sequences.append([char_to_idx[ch] for ch in target_chunk])

X = np.array(input_sequences)
y = np.array(target_sequences)

# -------------------------------
# 4. Build RNN Model
# -------------------------------
embedding_dim = 16
rnn_units = 64

inputs = Input(shape=(sequence_length,))
x = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
x = SimpleRNN(units=rnn_units, return_sequences=True)(x)
outputs = Dense(vocab_size, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# -------------------------------
# 5. Train the Model
# -------------------------------
model.fit(X, y, epochs=500, verbose=0)

# -------------------------------
# 6. Generate Sequence from Model
# -------------------------------
def generate_sequence(start_text, gen_length=100):
    generated = [char_to_idx[ch] for ch in start_text]
    for _ in range(gen_length):
        input_seq = np.array(generated[-sequence_length:]).reshape(1, -1)
        if input_seq.shape[1] < sequence_length:
            input_seq = np.pad(input_seq, ((0, 0), (sequence_length - input_seq.shape[1], 0)))
        pred = model.predict(input_seq, verbose=0)
        next_char = np.argmax(pred[0, -1])
        generated.append(next_char)
    return ''.join([idx_to_char[i] for i in generated])

# -------------------------------
# 7. Run it!
# -------------------------------
start_text = "hel"  # can be any part of the sentence
generated_text = generate_sequence(start_text, gen_length=10)
print(f"\n📝 Generated text:\n{generated_text}")



📝 Generated text:
hello how are
